In [1]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import joblib
import os

In [2]:
df1 = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\01_traingroupby.csv")
df1 = addcolumns(df1)
def combine_weather_10min(df1:pd.DataFrame):
    weather_df = pd.read_csv(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\10mins_weather_data.csv")
    weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
    weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
    weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間","時段"],
        right_on=["日期","時間","時段"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
comdf = combine_weather_10min(df1)
comdf.isna().sum()
comdf.loc[comdf["UVIndex"].isna(),["日期","時間","時段"]]

,日期,時間,時段
337,2024-01-06,06,20
338,2024-01-06,06,30
339,2024-01-06,06,40
340,2024-01-06,06,50
341,2024-01-06,07,00
...,...,...,...
10268,2024-07-14,18,20
10269,2024-07-14,18,30
10270,2024-07-14,18,40
10271,2024-07-14,18,50


In [3]:
df1 = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\01_traingroupby.csv")
df1 = addcolumns(df1)
def combine_weather(df1:pd.DataFrame):
    weather_df = pd.read_csv(r"D:\KPI報告\ml_power\station466990weather.csv")
    weather_df = weather_df.loc[:,["DateTime","TX01","GR01","RH01","UV01"]]
    weather_df["DateTime"] = weather_df["DateTime"].astype(str)
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].str.slice(0,8))
    weather_df["時間"] = weather_df["DateTime"].str.slice(8,10)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間"],
        right_on=["日期","時間"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
combine_weather(df1)

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,時間,時段,位置,DateTime,TX01,GR01,RH01,UV01
0,20240101063001,1,2024-01-01 06:30:00,0.0,1016.544444,17.50,86.777778,50.093333,0.002222,2024-01-01,06,30,01,2024010106,18.7,0.00,63,0.00
1,20240101064001,1,2024-01-01 06:40:00,0.0,1016.589000,17.49,86.800000,163.250000,0.016000,2024-01-01,06,40,01,2024010106,18.7,0.00,63,0.00
2,20240101065001,1,2024-01-01 06:50:00,0.0,1016.660000,17.50,86.950000,326.834000,0.046000,2024-01-01,06,50,01,2024010106,18.7,0.00,63,0.00
3,20240101070001,1,2024-01-01 07:00:00,0.0,1016.730000,17.50,86.990000,470.834000,0.083000,2024-01-01,07,00,01,2024010107,18.8,0.01,64,0.00
4,20240101071001,1,2024-01-01 07:10:00,0.0,1016.774000,17.57,86.970000,659.667000,0.139000,2024-01-01,07,10,01,2024010107,18.8,0.01,64,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10691,20240724174001,1,2024-07-24 17:40:00,0.0,961.297000,25.22,100.000000,91.251000,0.002000,2024-07-24,17,40,01,2024072417,27.0,0.02,88,0.04
10692,20240724175001,1,2024-07-24 17:50:00,0.0,961.056000,24.91,100.000000,53.999000,0.000000,2024-07-24,17,50,01,2024072417,27.0,0.02,88,0.04
10693,20240724180001,1,2024-07-24 18:00:00,0.0,960.907000,24.76,100.000000,37.248000,0.000000,2024-07-24,18,00,01,2024072418,27.1,0.01,92,0.01
10694,20240724181001,1,2024-07-24 18:10:00,0.0,959.856000,24.97,100.000000,40.332000,0.000000,2024-07-24,18,10,01,2024072418,27.1,0.01,92,0.01


In [4]:
def selecttime(df:pd.DataFrame):
    df["TimeGroup"] = pd.to_datetime(df["TimeGroup"])
    df = df[(df["TimeGroup"].dt.hour >= 9) & (df["TimeGroup"].dt.hour < 17)| ((df["TimeGroup"].dt.hour == 17) & (df["TimeGroup"].dt.minute == 0))]
    return(df)
selecttime(combine_weather(df1))

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,時間,時段,位置,DateTime,TX01,GR01,RH01,UV01
15,20240101090001,1,2024-01-01 09:00:00,1.91588,1017.906000,18.33,92.29,8395.250000,17.368000,2024-01-01,09,00,01,2024010109,18.8,0.19,64,0.00
16,20240101091001,1,2024-01-01 09:10:00,0.09000,1017.965000,18.02,97.37,7046.501000,12.612000,2024-01-01,09,10,01,2024010109,18.8,0.19,64,0.00
17,20240101092001,1,2024-01-01 09:20:00,0.01800,1018.163000,17.73,99.93,5919.584000,8.807000,2024-01-01,09,20,01,2024010109,18.8,0.19,64,0.00
18,20240101093001,1,2024-01-01 09:30:00,0.01800,1018.205000,17.90,100.00,8838.918000,17.564000,2024-01-01,09,30,01,2024010109,18.8,0.19,64,0.00
19,20240101094001,1,2024-01-01 09:40:00,0.03600,1018.131000,18.08,100.00,5774.665000,7.508000,2024-01-01,09,40,01,2024010109,18.8,0.19,64,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10683,20240724162001,1,2024-07-24 16:20:00,0.00000,959.121000,25.10,100.00,758.085000,0.106000,2024-07-24,16,20,01,2024072416,25.6,0.07,95,0.22
10684,20240724163001,1,2024-07-24 16:30:00,0.00000,959.102000,25.10,100.00,507.167000,0.054000,2024-07-24,16,30,01,2024072416,25.6,0.07,95,0.22
10685,20240724164001,1,2024-07-24 16:40:00,0.00000,958.566667,25.10,100.00,369.998889,0.025556,2024-07-24,16,40,01,2024072416,25.6,0.07,95,0.22
10686,20240724165001,1,2024-07-24 16:50:00,0.00000,958.926000,25.16,100.00,336.249000,0.020000,2024-07-24,16,50,01,2024072416,25.6,0.07,95,0.22


In [10]:
merge_df.columns

Index(['Serial_x', 'LocationCode_x', 'TimeGroup_x', 'WindSpeed(m/s)_x',
       'Pressure(hpa)_x', 'Temperature(°C)_x', 'Humidity(%)_x',
       'Sunlight(Lux)_x', 'Power(mW)_x', '日期', '時間', '時段', '位置_x',
       'DateTime_x', 'TX01_x', 'GR01_x', 'RH01_x', 'UV01_x', 'StationId_x',
       'DateTime_r_x', 'SunshineDuration_x', 'UVIndex_x', 'AirTemperature_x',
       'DailyExtremeHighAirTemperature_x', 'DailyExtremeLowAirTemperature_x',
       'RelativeHumidity_x', 'VisibilityDescription_x', 'StationLatitude_x',
       'StationLongitude_x', 'StationAltitude_x', 'Serial_y', 'LocationCode_y',
       'TimeGroup_y', 'WindSpeed(m/s)_y', 'Pressure(hpa)_y',
       'Temperature(°C)_y', 'Humidity(%)_y', 'Sunlight(Lux)_y', 'Power(mW)_y',
       '位置_y', 'DateTime_y', 'TX01_y', 'GR01_y', 'RH01_y', 'UV01_y',
       'StationId_y', 'DateTime_r_y', 'SunshineDuration_y', 'UVIndex_y',
       'AirTemperature_y', 'DailyExtremeHighAirTemperature_y',
       'DailyExtremeLowAirTemperature_y', 'RelativeHumidity_y',

In [11]:
d = {}
for i in range(1,18):
    for j in range(1,18):
        if i==j:
            continue
        l1 = "0" + str(i) if i<10 else str(i)
        l2 = "0" + str(j) if j<10 else str(j)
        df1 = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\{l1}_traingroupby.csv")
        df2 = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\{l2}_traingroupby.csv")
        df1 = addcolumns(df1)
        df1 = combine_weather(df1)
        df1 = combine_weather_10min(df1)
        df1 = selecttime(df1)
        df2 = addcolumns(df2)
        df2 = combine_weather(df2)
        df2 = selecttime(df2)
        df2 = combine_weather_10min(df2)
        merge_df = pd.merge(
            df1,
            df2,
            left_on=['日期', '時間', '時段'],    # 左表連接的鍵
            right_on=['日期', '時間', '時段'], # 右表連接的鍵
            how='inner',
            suffixes=["_x","_y"]                         # 左表有x 代表我們要用左觀測站的資料來預測右觀測站的power
        )

        X = merge_df.loc[:,["WindSpeed(m/s)_x","Pressure(hpa)_x", "Temperature(°C)_x","Humidity(%)_x","Sunlight(Lux)_x",
                            "TX01_x","GR01_x","RH01_x","UV01_x",
                            "SunshineDuration_x","UVIndex_x",
                            "Power(mW)_x"]].reset_index(drop=True)
        Y = merge_df.loc[:,["Power(mW)_y"]].reset_index(drop=True)
        Y = Y.values.ravel()
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

        model = RandomForestRegressor(n_jobs=14)
        scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
        MAE = -sum(scores)
        model.fit(X_train, y_train)
       
        model_file_name = f'{l2}_{l1}.pkl'  #l2的模型鐘用l1預測的
        model_floder_name = "train_use_full_data_two_weather"         #將模型相關資訊儲存在這個檔案下
        model_file_path = os.path.join(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location",model_floder_name,model_file_name)
        os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
        joblib.dump(model, model_file_path)
        print(model_file_name ," MAE:",MAE )
        print(f"模型已保存至: {model_file_path}")
        if l2 not in d:
            d[l2]={}
        d[l2][model_file_name] =MAE

02_01.pkl  MAE: 146.68170339737276
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_two_weather\02_01.pkl
03_01.pkl  MAE: 191.46997571233538
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_two_weather\03_01.pkl
04_01.pkl  MAE: 190.67804347316428
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_two_weather\04_01.pkl
05_01.pkl  MAE: 257.5896308461312
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_two_weather\05_01.pkl
06_01.pkl  MAE: 227.74846662647107
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_two_weather\06_01.pkl
07_01.pkl  MAE: 168.75070334414212
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location

# 紀錄每個model的MAE並且存成json檔案

In [12]:
import json
json_output = json.dumps(d,indent=4)
output_path = f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\model_each_location\\{model_floder_name}\\d.json"
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json_output)

In [6]:
import json
model_floder_name = "train_use_full_data_two_weather" 
with open(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\model_each_location\\{model_floder_name}\\d.json", "r", encoding="utf-8") as file:
    mae_data = json.load(file)

In [7]:
#讀入json檔案  必且將每個模型的mae由小到大依序排序
for k in mae_data.keys():
    mae_data[k] = dict(sorted(mae_data[k].items(), key=lambda item: item[1]))
mae_data

{'02': {'02_07.pkl': 126.20490050264547,
  '02_01.pkl': 146.68170339737276,
  '02_03.pkl': 153.00495045989823,
  '02_06.pkl': 160.62997387887205,
  '02_04.pkl': 173.20815991045362,
  '02_05.pkl': 224.9175777961059,
  '02_14.pkl': 244.04120114372347,
  '02_13.pkl': 267.29140789293047,
  '02_10.pkl': 291.1522549655758,
  '02_09.pkl': 354.13508822463245,
  '02_08.pkl': 439.3539210132357,
  '02_12.pkl': 454.81303422706253,
  '02_11.pkl': 478.97735718018583,
  '02_16.pkl': 600.1993852343576,
  '02_15.pkl': 632.1077240743655,
  '02_17.pkl': 679.3223491945812},
 '03': {'03_02.pkl': 161.5290130901041,
  '03_04.pkl': 187.5515897533423,
  '03_01.pkl': 191.46997571233538,
  '03_06.pkl': 230.1557663515802,
  '03_07.pkl': 233.47442379029866,
  '03_05.pkl': 254.3096350120621,
  '03_10.pkl': 310.6806317685812,
  '03_13.pkl': 363.1528976532535,
  '03_14.pkl': 371.8163472571073,
  '03_11.pkl': 451.971435331542,
  '03_09.pkl': 473.2925545241678,
  '03_12.pkl': 565.2899989419767,
  '03_08.pkl': 600.45889

In [8]:
#將不同觀測站的資料load近來並放在字典裡
df_dict = {}
for i in range(1,18):
    location = "0" + str(i) if i<10 else str(i)
    df = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\{location}_traingroupby.csv")
    df = addcolumns(df)
    df = combine_weather(df)
    df = combine_weather_10min(df)
    df_dict[location] = df

In [9]:
#將不同模型load近來並放在字典裡
model_dict = {}
for k1 in mae_data.keys():
    for model_name in mae_data[k1].keys():
        model_file_path = os.path.join(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location",model_floder_name,model_name)
        model_dict[model_name] = joblib.load(model_file_path)

In [10]:
#將檔案讀進來  新增日期時間位置等欄位以便我們找資料對應
DataName = r"D:\KPI報告\ml_power\36_TestSet_SubmissionTemplate\upload(no answer).csv"
ans = pd.read_csv(DataName, encoding='utf-8')
ans["序號"] = ans["序號"].astype(str)
ans["日期"] = pd.to_datetime(ans["序號"].str.slice(0,8))
ans["時間"] =ans["序號"].str.slice(8,10)
ans["時段"] =ans["序號"].str.slice(10,12)
ans["位置"] =ans["序號"].str.slice(12,14)
ans["模型順位"] = None
ans["使用模型MAE"] = None
for i in range(len(ans)):
# for i in range(3):  
    date,time,timecount,ans_location = ans.loc[i,"日期"],ans.loc[i,"時間"],ans.loc[i,"時段"],ans.loc[i,"位置"]  #找出答案該個row的時期時間時段以及位置
    mae_table = mae_data[ans_location]   #找出這個位置的所有模型(來自另外16個模型) 並且已按照MAE順序排
    j = 1
    for model_name in mae_table.keys():  #從MAE最小的模型(觀測站)去找有沒有該觀測站的資料  如果有就用該資料跟模型來進行predict
        X_location = model_name[3:5]     # X_locatioan是訓練集模型的地點
        df_sametimedata = df_dict[X_location]
        # df_sametimedata = addcolumns(df_sametimedata)
        # df_sametimedata = combine_weather(df_sametimedata)
        search_df = df_sametimedata.loc[(df_sametimedata["日期"]==date) & (df_sametimedata["時間"]==time) & (df_sametimedata["時段"]==timecount),]
        if len(search_df)==1:  #如果有找資料 就用這個模型predict 並且break迴圈  沒有就繼續往MAE較大的模型找 找到都沒有則答案欄位就保持NA
            X_test = search_df.loc[:,["WindSpeed(m/s)","Pressure(hpa)", "Temperature(°C)","Humidity(%)","Sunlight(Lux)",
                            "TX01","GR01","RH01","UV01",
                            "SunshineDuration","UVIndex",
                            "Power(mW)"]]  
            X_test = X_test.rename(columns=lambda col: f"{col}_x")  #將每個變數都加上_x以符合train那邊的形式
            model = model_dict[model_name]
            predictions = model.predict(X_test)
            ans.loc[i,"答案"] = predictions[0]
            ans.loc[i,"模型順位"] = j
            ans.loc[i,"使用模型MAE"] = mae_table[model_name]
            break
        j = j+1 
    if i %100==0:
        print(i)

            

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500


In [11]:
ans_nona = ans.loc[~ans["答案"].isna(),].reset_index(drop=True)
ans_na = ans.loc[ans["答案"].isna(),].reset_index(drop=True)
ans_nona.to_csv("ans_nona.csv",index=False)

In [12]:
ans_nona.loc[ans["使用模型MAE"]>=700,]

,序號,答案,日期,時間,時段,位置,模型順位,使用模型MAE


In [13]:
# 清空字典
import gc
model_dict.clear()

# 強制執行垃圾回收
gc.collect()

print("所有模型已釋放")

所有模型已釋放


In [14]:
model_lastday_floder_name = "train_lastday_gridsearch_usealldata"
with open(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\model_each_location\\{model_lastday_floder_name}\\d_lastday.json", "r", encoding="utf-8") as file:
    mae_lastday_data = json.load(file)

In [15]:
model_lastday_dict = {}
for k1 in mae_lastday_data.keys():
    model_name = f"{k1}" 
    model_file_path = os.path.join(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location",model_lastday_floder_name,model_name)
    model_lastday_dict[k1] = joblib.load(model_file_path)

In [26]:
df_dict["01"].columns

Index(['Serial', 'LocationCode', 'TimeGroup', 'WindSpeed(m/s)',
       'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)',
       'Power(mW)', '日期', '時間', '時段', '位置', 'DateTime', 'TX01', 'GR01', 'RH01',
       'UV01', 'StationId', 'DateTime_r', 'SunshineDuration', 'UVIndex',
       'AirTemperature', 'DailyExtremeHighAirTemperature',
       'DailyExtremeLowAirTemperature', 'RelativeHumidity',
       'VisibilityDescription', 'StationLatitude', 'StationLongitude',
       'StationAltitude'],
      dtype='object')

In [17]:
#當答案欄位有空格  就會去找MAE最小的模型  並且用前一天的資料去predict 沒前一天的資料就找更前一天
for i in range(len(ans_na)):
# for i in range(3):  
    if i%10==0:
        print(i)
    time,timecount,ans_location = ans_na.loc[i,"時間"],ans_na.loc[i,"時段"],ans_na.loc[i,"位置"]
    # mae_table = mae_data[ans_location]
    # model_name = list(mae_table.keys())[0]  
    # X_location = model_name[3:5]
    df_sametimedata = df_dict[ans_location]
    df_sametimedata = addcolumns(df_sametimedata)
    days = 1
    while True:
        date = ans_na.loc[i,"日期"] - pd.Timedelta(days=days)
        search_df = df_sametimedata.loc[(df_sametimedata["日期"]==date) & (df_sametimedata["時間"]==time) & (df_sametimedata["時段"]==timecount),]
        if len(search_df)==1:
            X_test = search_df.loc[:,['WindSpeed(m/s)', 'Pressure(hpa)','Temperature(°C)','Humidity(%)', 'Sunlight(Lux)', 
                                      'TX01','GR01','RH01','UV01','Power(mW)']]  
            X_test = X_test.rename(columns=lambda col: f"{col}_x")
            model = model_lastday_dict[f"{ans_location}.pkl"]

            
            predictions = model.predict(X_test)
            ans_na.loc[i,"答案"] = predictions[0]
            break
        days = days + 1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [18]:
res = pd.concat([ans_nona,ans_na],axis=0).reset_index(drop=True)
res.loc[:,["序號","答案"]].to_csv("1128ans5_alldata_2weather_data.csv",index=False)

In [11]:
merge_df.loc[:,["Temperature(°C)_x","Humidity(%)_x","Sunlight(Lux)_x","Power(mW)_x","Temperature(°C)_y","Humidity(%)_y","Sunlight(Lux)_y","Power(mW)_y"]].corr()

,Temperature(°C)_x,Humidity(%)_x,Sunlight(Lux)_x,Power(mW)_x,Temperature(°C)_y,Humidity(%)_y,Sunlight(Lux)_y,Power(mW)_y
Temperature(°C)_x,1.000000,-0.865882,0.713088,0.663425,0.985160,-0.743495,0.668910,0.649906
Humidity(%)_x,-0.865882,1.000000,-0.722009,-0.683149,-0.844816,0.902799,-0.704469,-0.677605
Sunlight(Lux)_x,0.713088,-0.722009,1.000000,0.983972,0.726162,-0.761528,0.982015,0.979144
Power(mW)_x,0.663425,-0.683149,0.983972,1.000000,0.678282,-0.720957,0.973662,0.991635
Temperature(°C)_y,0.985160,-0.844816,0.726162,0.678282,1.000000,-0.744065,0.688686,0.668247
Humidity(%)_y,-0.743495,0.902799,-0.761528,-0.720957,-0.744065,1.000000,-0.740842,-0.714182
Sunlight(Lux)_y,0.668910,-0.704469,0.982015,0.973662,0.688686,-0.740842,1.000000,0.983198
Power(mW)_y,0.649906,-0.677605,0.979144,0.991635,0.668247,-0.714182,0.983198,1.000000
